In this example we want to convert our tables to json schemas.
It should move us forward into preparing good culture of our models json schema reflection.
We could validate the data and provide documentation based on this data.

In [ ]:
%pip install openai

In [121]:
# Importing the necessary Python libraries
import os
import re
import json
import yaml
import openai
import time

In [4]:
# Loading the API key and organization ID from file (NOT pushed to GitHub)
with open('../keys/openai_creds.yaml') as f:
    keys_yaml = yaml.safe_load(f)

# Applying our API key and organization ID to OpenAI
openai.organization = keys_yaml['ORG_ID']
openai.api_key = keys_yaml['API_KEY']
os.environ['OPENAI_API_KEY'] = keys_yaml['API_KEY']

In [ ]:
# Find all tables
# Loading the SQL Structure text from local file
with open('../schemas/structure.sql', 'r') as f:
    sql_structure = f.read()

print(sql_structure)

In [64]:
# Searching for all tables in sql structure
table_patterns = re.findall(r'CREATE TABLE ([\w\.]+) \(([\s\S]*?)\)\;', sql_structure)

In [ ]:
# Engineering a prompt to extract table information from structure as a JSON object
prompt = f'''
You are the most talented engineer with more than half of century of experience. 
Please extract information from the sql structure provided and convert it to a JSON schema object.

This is the body of text to extract the information from:

table name: {table_name}

table fields: {table_fields}
'''

In [ ]:
def get_prompt(table_name, table_fields):
    return f'''
    You are the most talented engineer with more than half of century of experience. 
    Please extract information from the sql structure provided and convert it to a JSON schema object.
    Please write description for every field if it's possible to understand from the context.
    The root key of the json schema should be table name.
    Please strictly stick to OpenApi schema structure.
    Assume that all the fields are required by default.
    This is the body of text to extract the information from:
    
    table name: {table_name}
    
    table fields: {table_fields}
    '''

In [ ]:
def chatgpt_call(prompt):
    return openai.ChatCompletion.create(
        model = 'gpt-3.5-turbo',
        messages = [{'role': 'user', 'content': prompt}]
    )

In [ ]:
def write_json_to_file(json_response, filename='output.json'):
    try:
        # Write the parsed JSON back to the file
        with open(filename, 'w') as file:
            json.dump(json_response, file, indent=4)
        
        print(f"Data successfully written to {filename}")

    except json.JSONDecodeError:
        print("Error: The provided string is not valid JSON.")


In [ ]:
# for table_pattern in table_patterns:
for table_pattern in table_patterns[:5]:
    # Table name group
    table_name = table_pattern[0]
    # Field names group
    table_fields = table_pattern[1]

    prompt = get_prompt(table_name, table_fields)

    chatgpt_response = chatgpt_call(prompt)
    
    # Loading the response as a JSON object
    json_response = json.loads(chatgpt_response['choices'][0]['message']['content'])

    path = os.path.join("schemas/jsons/", table_name.replace('.', '_') + ".json")

    write_json_to_file(json_response, path)

    time.sleep(0.5)